In [ ]:
#Useufl Libraries
from IPython.display import display
from tabulate import tabulate
import pandas as pd
# We'll also import seaborn, a Python graphing library
import warnings # current version of seaborn generates a bunch of warnings that we'll ignore
warnings.filterwarnings("ignore")
import seaborn as sns

%matplotlib inline
import matplotlib.pyplot as plt
plt.style.use('ggplot')
sns.set(style="white", color_codes=True)

**Some Basic Data Processing First**
====================================

In [ ]:
#Working folder
data_dir = "../data/"
output_dir = '../outputs/'
d_file_v0 = data_dir + "data_v0.csv"

dates = ["T! Ship arrival",
         "T2 Manifest received",
         "T3 Declaration lodged",
         "T4 Declaration validated",
         "T5 Physical inspection",
         "T6 Clearance",
         "T7 Exit"
        ]
d = pd.read_csv(d_file_v0, parse_dates=dates)

In [ ]:
#Rename columns
old_names = d.columns
new_names= ['declaration_yr','declaration_num','manifest_yr',
            'manifest_num','ref_ship_service','t1_arrival',
            't2_manifest_recvd','t3_declaration_lodged',
            't4_declaration_validated','t5_inspection',
            't6_clearance','t7_removal']
d.rename(columns=dict(zip(old_names, new_names)), inplace=True)

In [ ]:
#Drop cases where the sequence isnt T7>T6>T5>T4>T3>T2>T1
#So basically, if one of the time stamps is missing
len_all = d.shape[0]
d.dropna(axis=0,
         how='any',
         subset=['t1_arrival',
            't2_manifest_recvd','t3_declaration_lodged',
            't4_declaration_validated','t5_inspection',
            't6_clearance','t7_removal'],inplace=True)

print ("%s rows dropped"%(len_all-len(d)))

In [ ]:
#Any duplicates on d['declaration_num']
dup_decl = d['declaration_num'].duplicated()
print ("%s duplicates based on number declaration"%(len (dup_decl[dup_decl==True])))

dup_mani = d['manifest_num'].duplicated()
print ("%s duplicates based on number manifest"%(len (dup_mani[dup_mani==True])))

In [ ]:
#len (dup[dup==False])
len(d)

**Add variables to represent the three different phases**
=========================================================
1. t1-start of this phase: 't1_arrival'
2. t2-start of this phase:  't3_declaration_lodged'
3. t3-start of this phase: 't7_removal'

In [ ]:
#Add the phases
d['t1'] = d.t1_arrival
d['t2'] = d.t3_declaration_lodged
d['t3'] = d.t7_removal

**Add date related variables for all time-stamps**
===================================================
1. date
2. month
3. day
4. day of the week
5. hr
6. week dayr or weekend

In [ ]:
#Creates a binary variable to see whether this data is a weekday(or holiday) or week/working day
#Hoilday list generated from: http://www.timebie.com/calendar/algeria2016.php
holidays = ['2015-12-24','2015-11-01','2015-10-13','2015-09-25',
           '2015-09-24','2015-06-19','2016-12-12','2016-11-01',
           '2016-10-12','2016-09-14','2016-07-07','2016-07-05',
           '2016-06-19','2016-05-01','2016-01-01']

def week_end(row,column):
    if row[column +"_" + 'wkday_name']=='Friday' or row[column +"_" + 'wkday_name']=='Saturday':
        return 1
    elif row[column +"_"+'date'] in holidays:
        return 1
    else:
        return 0

In [ ]:
#For all the time-stamp variables add the date related variables
def add_dates_vars (column, df):
    d[column +"_" + 'yr'] = d.apply(lambda row: row[column].year,axis=1)
    d[column +"_" + 'date'] = d.apply(lambda row: row[column].date().strftime('%Y-%m-%d'),axis=1)
    d[column +"_" + 'month'] = d.apply(lambda row: row[column].month,axis=1)
    d[column +"_" + 'day'] = d.apply(lambda row: row[column].day,axis=1)
    d[column +"_" + 'wkday'] = d.apply(lambda row: row[column].weekday(),axis=1)
    d[column +"_" + 'wkday_name'] = d.apply(lambda row: row[column].weekday_name,axis=1)
    d[column +"_" + 'hr'] = d.apply(lambda row: row[column].hour,axis=1)
    d[column +"_" + 'wkend']= d.apply(lambda row: week_end(row, column), axis=1)

In [ ]:
#Now add all the date variables
dates = ['t1_arrival','t2_manifest_recvd','t3_declaration_lodged',
            't4_declaration_validated','t5_inspection',
            't6_clearance','t7_removal', 't1', 't2','t3']
for col in dates:
    #add_dates_vars(col,d)
    add_dates_vars (col, d)

In [ ]:
#Add duration variables-use exact time stamps to compute these and save as total seconds
#Main phases
d['t3_t1'] = d.apply(lambda row: (row['t3_declaration_lodged']-row['t1_arrival']).total_seconds(),axis=1)
d['t6_t3'] = d.apply(lambda row: (row['t6_clearance']-row['t3_declaration_lodged']).total_seconds(),axis=1)
d['t7_t6'] = d.apply(lambda row: (row['t7_removal']-row['t6_clearance']).total_seconds(),axis=1)

#Add othger time steps
d['t6_t5'] = d.apply(lambda row: (row['t6_clearance']-row['t5_inspection']).total_seconds(),axis=1)
d['t5_t4'] = d.apply(lambda row: (row['t5_inspection']-row['t4_declaration_validated']).total_seconds(),axis=1)
d['t4_t3'] = d.apply(lambda row: (row['t4_declaration_validated']-row['t3_declaration_lodged']).total_seconds(),axis=1)
d['t3_t2'] = d.apply(lambda row: (row['t3_declaration_lodged']-row['t2_manifest_recvd']).total_seconds(),axis=1)
d['t2_t1'] = d.apply(lambda row: (row['t3_declaration_lodged']-row['t2_manifest_recvd']).total_seconds(),axis=1)

In [ ]:
#Only considering the sequences-Add step with maximum and minimum duration
durations = ['t7_t6','t6_t5','t5_t4','t4_t3','t3_t2','t2_t1' ]
d['max_step_duration'] = d[durations].idxmax(axis=1)
d['min_step_duration'] = d[durations].idxmin(axis=1)

#Also find out which phases takes the longest time-this could be obvious due to the number of steps at phase
d['max_phases'] = d[['t7_t6',"t3_t1",'t6_t3']].idxmax(axis=1)
d['min_phases'] = d[['t7_t6',"t3_t1",'t6_t3']].idxmin(axis=1)

**We can quickly check how the distribution is for maximum duration**

The results show that step T3-T2 is the one which takes the longest time.

In [ ]:
#Plot to show which step takes the longest time
d.max_step_duration.value_counts().plot(kind='bar', 
                                        title ="Steps with maximum duration", 
                                        figsize=(15, 10), legend=True, fontsize=12)

**Which phase takes the longest time?**
===========================================
Surprisingly (may be it not) the upstream phase seem to take the longest time (as shown in plot below)
although it has few steps

In [ ]:
#Plot to show which phase takes the longest time
d.max_phases.value_counts().plot(kind='bar', 
                                        title ="Phase with maximum duration", 
                                        figsize=(15, 10), legend=True, fontsize=12)

**Compute  basic statistics for the sample**
============================================
In all this, the assumption is that the data is independent and identuically distributed. 
Essentially, arrival of one container doesnt affect the other BUT this could be wrong. For
instance, if containers arrive in a batch, they may choose to process them togather.

1. Container arrivals-just see count of containers by date, day of the week
2. 

In [ ]:
df_date = pd.DataFrame(d.t1_arrival_date.value_counts())
df_date.index = pd.DatetimeIndex(df_date.index)
df_date.plot(figsize=(12,8))

In [ ]:
#Arrival time by month
d.t1_arrival_month.value_counts().plot(kind='bar', 
                                        title ="Arrival month", 
                                        figsize=(15, 10), legend=True, fontsize=12)

**Distribution of Day of the week and time (essentially hour) of the day or each time stamp**
================================================================================================
1. Table showing proportions across each day/hour
2. Barplot
3. Inferring distributions-although this isnt easy for categorical variables

**In the next cells I output tables for day and hourly distribution......** 

In [188]:
#Function to combine time-stamp columns into a single table for easy comparison
def create_table(cat,df_data):
    i=0
    for dt in dates:
        if i==0:
            df = pd.DataFrame(df_data[dt + "_" + cat].value_counts()).reindex(['Sunday','Monday','Tuesday','Wednesday','Thursday','Friday','Saturday']) 
        else:
            df = pd.concat([df, pd.DataFrame(df_data[dt + "_" + cat].value_counts()).reindex(['Sunday','Monday','Tuesday','Wednesday','Thursday','Friday','Saturday'])],axis=1, join='inner')
        i += 1
    return df

In [190]:
#Compute and save the Frequencies
output_dir = '../outputs/'
days = create_table('wkday_name', d)
days.to_csv(output_dir + 'wkday_freqs.csv', index=True,header=True)
hrs = create_table('hr', d)
hrs.to_csv(output_dir + 'hr_freqs.csv', index=True,header=True)

In [187]:
days

,t1_arrival_wkday_name,t2_manifest_recvd_wkday_name,t3_declaration_lodged_wkday_name,t4_declaration_validated_wkday_name,t5_inspection_wkday_name,t6_clearance_wkday_name,t7_removal_wkday_name,t1_wkday_name,t2_wkday_name,t3_wkday_name
Sunday,3123,3243,3423,3417.0,3412.0,3174,2912,3123,3423,2912
Monday,3070,2696,3748,3377.0,3382.0,3392,3185,3070,3748,3185
Tuesday,2803,3007,3656,3438.0,3436.0,3527,3413,2803,3656,3413
Wednesday,3617,2974,3423,3782.0,3784.0,3724,3649,3617,3423,3649
Thursday,3497,4721,3646,3895.0,3895.0,4089,3978,3497,3646,3978
Friday,1081,138,5,NaN,NaN,5,64,1081,5,64
Saturday,727,1139,17,9.0,9.0,7,717,727,17,717
